<a href="https://colab.research.google.com/github/ericslevenson/arctic-surface-water/blob/main/S2bgrnExport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
author: ericslevenson

Earth Engine Script to rapidly filter and visualize Sentinel-2 images to provide
image inputs to the UNET component of LakeScan.

NOTE: images are either exported from GEE directly to drive, or the 'PRODUCT_ID' 
can be recorded for batch downloads
'''

In [ ]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
from google.colab import auth 
auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize() # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

# Some common imports
from IPython.display import Image
import folium

In [ ]:
## ***IMAGE PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

# Get percentile cover   
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actArea = ee.Number(image.updateMask(image.select('B2')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      maxPixels=1e12,
      ).values().get(0)).multiply(10000)
  # calculate the perc of cover OF CLEAR PIXELS 
  percCover = actArea.divide(area).multiply(100)
  # number as output
  return image.set('percCover', percCover,'actArea',actArea)

# Select bgrn bands
def selectBands(image):
  img = image.select(['B2', 'B3', 'B4', 'B8'])
  return img

def getDims(image):
  dimensions = img.select('B2').getInfo().get('bands')[0].get('dimensions')
  return image.set('dims', dimensions)

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [220]:
# *** User inputs to customize Sentinel-2 Tile (i.e. roi), time period, and clear sky threshold ***
tiles = ee.FeatureCollection('projects/ee-eric-levenson/assets/ArcticAKaois/S2Tiles_AK_4N') # import Sentinel-2 tiles as a GEE Feature Collection
tile = '04WDA' # Tile of interest...this defines the ROI
roi = tiles.filter(ee.Filter.eq('Name', tile)).first() 
start = '2016-05-10'
finish = '2016-09-15'
# Customize month range of interest (1 = January, etc.)
startMonth = 6
finishMonth = 8
eestart = ee.Date(start)
eefinish = ee.Date(finish)
# clear roi percentage threshold
coverage = 90 

## ***Image Processing...don't edit***
images = ee.ImageCollection('COPERNICUS/S2').filterBounds(roi.geometry()).filter(ee.Filter.equals('MGRS_TILE',tile)).filterDate(start,finish).filter(ee.Filter.calendarRange(startMonth, finishMonth, 'month')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = images.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(clip_image) # Clip to roi
images_all = images_all.map(applyMask) # mask other bands for clouds
area = roi.geometry().area().getInfo() # Calculate total area
images_all = images_all.map(getCover) # Calculate percent cover and add as an image property
images_all = images_all.filterMetadata('percCover','greater_than', coverage) # remove images covering less than given threshold)
images_all = images_all.map(selectBands) # reduce to bgrn bands
percCovers = images_all.aggregate_array('percCover').getInfo() # get non EE list of percent covers
maxim = max(percCovers) # get maximum percent cover
best_image = images_all.filterMetadata('percCover','equals',maxim) # get image with max percent cover
img = ee.Image(best_image.first())
date = img.getInfo()['id'].split('/')[2].split('_')[0][:8] # get date for export
filename = str(date+'_'+tile+'_BGRN_SR') # create filename string

In [221]:
percCovers # Print the list of coverage percentages for the time period

[90.76783174515893,
 90.76783174515893,
 98.35584751159762,
 98.35584751159762,
 95.19059241142726,
 95.19059241142726,
 100.25316443162853,
 100.25316443162853]

In [222]:
filename # Print the filename of the clearest image, which includes the date

'20160829_04WDA_BGRN_SR'

In [223]:
## Visualize to double check ##

# Define the visualization parameters.
image_viz_params = {
    'bands': ['B8', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': [0.95, 1.1, 1]
}
# Define a map centered on San Francisco Bay.
map_l8 = folium.Map(zoom_start=10)
# Add the image layer to the map and display it.
map_l8.add_ee_layer(img, image_viz_params, 'true color composite')
#map_l8.add_ee_layer(image_mask, {'bands': ['B2']}, 'true color composite')
folium.GeoJson(roi.geometry().getInfo()).add_to(map_l8)
display(map_l8) 

In [ ]:
# IMAGE RETRIEVAL METHOD 1: GEE EXPORT TO DRIVE. Skip if batch downloading images.

task = ee.batch.Export.image.toDrive(**{
    'image': img,
    'description': filename,
    'folder':'S2_BGRN',
    'fileFormat': 'GeoTIFF',
    'scale': 10,
    'region': roi.geometry(),
    'maxPixels': 1e12
})
task.start()
import time 
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

In [224]:
# IMAGE RETRIEVAL METHOD 2: BATCH DOWNLOAD WITH SENTINELSAT API 

# Print product ID and record in ID_list below.
img.getInfo().get('properties').get('PRODUCT_ID') 

'S2A_MSIL1C_20160829T223532_N0204_R058_T04WDA_20160829T223532'

In [ ]:
# Record 'PRODUCT_ID' in ID_list for batch download
ID_list = ['S2A_MSIL1C_20210820T222541_N0301_R015_T04WFE_20210821T001506', 'S2A_MSIL1C_20160713T224532_N0204_R101_T04WDD_20160713T224534', 'S2A_MSIL1C_20160829T223532_N0204_R058_T04WDA_20160829T223532,' 'S2A_MSIL1C_20160713T224532_N0204_R101_T04WDB_20160713T224534', 'S2A_MSIL1C_20160713T224532_N0204_R101_T04WDC_20160713T224534', 'S2A_MSIL1C_20160829T223532_N0204_R058_T04WDV_20160829T223532', 'S2B_MSIL1C_20190707T222539_N0207_R015_T04WDS_20190707T235752', 'S2B_MSIL1C_20190707T222539_N0207_R015_T04WDT_20190707T235752', 'S2B_MSIL1C_20190707T222539_N0207_R015_T04WDU_20190707T235752', 'S2B_MSIL1C_20180930T222529_N0206_R015_T04WFD_20181001T000258', 'S2B_MSIL1C_20190707T222539_N0207_R015_T04WEA_20190707T235752', 'S2B_MSIL1C_20190707T222539_N0207_R015_T04WEB_20190707T235752', 'S2A_MSIL1C_20200719T223541_N0209_R058_T04WEC_20200720T002418', 'S2B_MSIL1C_20180930T222529_N0206_R015_T04WFC_20181001T000258', 'S2A_MSIL1C_20200821T224541_N0209_R101_T04WEE_20200822T004503', 'S2A_MSIL1C_20200821T224541_N0209_R101_T04WEE_20200822T004503', 'S2B_MSIL1C_20180917T221529_N0206_R115_T04WFB_20180918T001514', 'S2A_MSIL1C_20190818T221531_N0208_R115_T04WFA_20190818T235258', 'S2A_MSIL1C_20170927T221531_N0205_R115_T04WEV_20170927T221528', 'S2A_MSIL1C_20170927T221531_N0205_R115_T04WEU_20170927T221528', 'S2A_OPER_PRD_MSIL1C_PDMC_20160516T154530_R115_V20160515T221537_20160515T221537', 'S2A_OPER_PRD_MSIL1C_PDMC_20160516T160324_R115_V20160515T221537_20160515T221805']
# Set image download directory
output_dir = "/content/drive/MyDrive/S2_downloaded/"

In [ ]:
## Import sentinelsat api and authenticate

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

api = SentinelAPI('elevens2', '0380!nH0380!nH', 'https://scihub.copernicus.eu/dhus')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Batch download images
if ID_list is not None:
    
    # loop through elements in ID list
    for i, ID in enumerate(ID_list):

        try:
            
            # find product using known ID
            products = api.query(identifier=ID)
            
            # print download message for each product
            print("Downloading image ID: {} ({} of {})".format(ID, i+1, len(ID_list)))
            
            # download single image by known product ID
            api.download_all(products,output_dir)

        except:

            # print error message if download fails
            print("Could not download Sentinel-2 image with ID {}.".format(ID))
            
else:
    # print error message if none or both of ID_list and geojson_files_path have been set to equal None
    print("Product ID(s) and file path(s) have been provided. Please make either 'ID_list' or 'geojson_files_path' equal to None.")